In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls"
REPO_LOC = "/content/drive/MyDrive/Projects/rhexis-trajectory"

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.insert(0,f"{REPO_LOC}/Trajectory_Classification")
from utils import *
%load_ext autoreload
%autoreload 2

# Load data

In [ ]:
names, path_dfs, labels, sizes = load_all_pulls(DATA_LOC)

In [ ]:
path_dfs[0]

In [ ]:
data = np.stack([featurize_pull(pull) for pull in path_dfs], axis=0)
data[0]

In [ ]:
X, y = np.stack([featurize_pull(pull, 25) for pull in path_dfs], axis=0), np.array(labels)
X[0]

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Keep deterministic
np.random.seed(42)
sss = StratifiedShuffleSplit(1, test_size=.2)
train_ind, test_ind = next(sss.split(X, y))
X_train, X_test = X[train_ind], X[test_ind]
y_train, y_test = y[train_ind], y[test_ind]
# Print to check class balance
y_train, y_test

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = LogisticRegression(random_state=0, multi_class="multinomial", max_iter=1000)
pipe = make_pipeline(StandardScaler(), clf)
# print(cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy'))
print(cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy'))
# clf.fit(X_train, y_train)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf = QuadraticDiscriminantAnalysis()
pipe = make_pipeline(StandardScaler(), clf)
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')